https://www.kaggle.com/competitions/used-cars-price-prediction-22ds соревнование

Описание полей данных



1. Date: Год выпуска автомобиля.
2. Make: Марка автомобиля.
3. Model: издание автомобиля определенной марки.
4. Trim: Уровни отделки салона автомобиля — это просто разные версии модели.
5. Body: Тип кузова транспортного средства относится к форме и модели конкретной марки автомобиля.
6. Transmission: механизм, который передает мощность от двигателя к колесам.
7. VIN: идентификационный номер транспортного средства.
8. State: состояние, в котором автомобиль выставлен на аукцион.
9. Condition: Состояние автомобилей на момент аукциона.
10. Odometer: расстояние, пройденное автомобилем с момента выпуска.
11. Color: Цвет кузова автомобиля.
12. Interior: Цвет салона автомобиля.
13. Seller: Продавец автомобиля, автосалоны.
14. mmr: Рекорд рынка Manhiem, рыночная оценочная цена автомобилей.
15. sellingprice: цена, по которой автомобиль был продан на аукционе.
16. saledate: Дата продажи автомобиля.




План работы
1. Загрузка данных и библиотек для работы
2. Обработка датасетов
3. Изучение данных
4. Постановка задачи для ML
5. Изучение лучших моделей обучения
6. Тестирование всех моделей на тестовой выборке
7. выбор наилучшей модели
8. Вывод


###1. Загрузка данных и библиотек для работы

In [ ]:
import pandas as pd

In [ ]:
try:
  train_db = pd.read_csv('train.csv')
  test_db = pd.read_csv('test.csv')
except:
  train_db = pd.read_csv('drive/MyDrive/ЯПрактикум/train.csv')
  test_db = pd.read_csv('drive/MyDrive/ЯПрактикум//test.csv')


FileNotFoundError: ignored

In [ ]:
train_db.head(5)

In [ ]:
test_db.head(5)

In [ ]:
train_db.info()

In [ ]:
test_db.info()

**Вывод:**

###2. Обработка датасетов





In [ ]:
!pip install phik
import phik

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def base_analizez(data):
    ''' функция для изучения датасета'''

    print('Просмотр датасета:')
    display(data.head())

    print('\n')
    print('Общая информация об датасете:')
    print(data.info())

    print('\n')
    print('Проверка на наличие значений nan в датасете:')
    display(pd.DataFrame(round(
        data.isna().mean()*100,)).style.background_gradient('coolwarm'))

    print('\n')
    print('Изучение корреляции между данными')

    # Т.к. в датасете присутствует большое количество категориальных
    # переменных применим метрику Phik (φk) корреляции.

    phik_overview = data.phik_matrix()

    plot_correlation_matrix(phik_overview.values.round(2),
                        x_labels=phik_overview.columns,
                        y_labels=phik_overview.index,
                        vmin=0, vmax=1, color_map="Greens",
                        title=r"correlation $\phi_K$",
                        fontsize_factor=1,
                        figsize=(15, 8))
    plt.tight_layout()

In [ ]:
%%time
base_analizez(train_db)

In [ ]:
base_analizez(test_db)

In [ ]:
display(pd.DataFrame(round(test_db.isna().mean()*100,)).style.background_gradient('coolwarm'))
for i in test_db.columns:
  if test_db[i].isna().mean()*100 > 0:
    print(i)

In [ ]:
# вырезание пустых значений
# full_data = full_data[full_data['film_studio'].isna() == False]